In [2]:
import pickle
import pandas as pd
from collections import defaultdict

# For Josh: Remove duplicates

In [1]:
f = open("Ads10000_vaccine", "rb")
ads = pickle.load(f)
f.close()

In [20]:
f = open("Ads10000_election", "rb")
ads_election = pickle.load(f)
f.close()

In [24]:
df = pd.DataFrame.from_records(ads)
df['ad_creative_bodies'] = df['ad_creative_bodies'].str[0]
df=df.drop_duplicates(subset=['ad_creative_bodies'])
df=df.sample(frac=1)

In [26]:
df_election = pd.DataFrame.from_records(ads_election)
df_election['ad_creative_bodies'] = df_election['ad_creative_bodies'].str[0]
df_election=df_election.drop_duplicates(subset=['ad_creative_bodies'])
df_election=df_election.sample(frac=1)

In [27]:
df.to_csv('Ads_vaccine_sample.csv')

In [ ]:
df_election.to_csv('Ads_election_sample.csv')

# For emily: combine duplicates

In [8]:
f = open("Ads10000_election", "rb")
ads_emily_election = pickle.load(f)
f.close()

In [9]:
f = open("Ads10000_vaccine", "rb")
ads_emily_vaccine = pickle.load(f)
f.close()

In [10]:
df2 = pd.DataFrame.from_records(ads_emily_election)
df2=df2.drop(['id', 'ad_delivery_start_time', 'ad_delivery_stop_time', 'ad_creation_time'], axis=1)
df2 = df2.dropna(subset=['delivery_by_region'])
df2['ad_creative_bodies'] = df2['ad_creative_bodies'].str[0]
df2_grouped = df2.groupby('ad_creative_bodies').agg({'demographic_distribution':lambda x: list(x), 
                                       'delivery_by_region':lambda x: list(x), 
                                      'demographic_distribution':lambda x: list(x), 
                                      'estimated_audience_size':lambda x: list(x), 
                                      'spend': lambda x: list(x), 
                                      'impressions': lambda x: list(x)}).reset_index()

In [19]:
df2_grouped.head()

,ad_creative_bodies,demographic_distribution,delivery_by_region,estimated_audience_size,spend,impressions
0,,"[[{'percentage': '0.000876', 'age': '45-54', '...","[[{'percentage': '1', 'region': 'New York'}], ...","[{'lower_bound': '10001', 'upper_bound': '5000...","[{'lower_bound': '300', 'upper_bound': '399'},...","[{'lower_bound': '8000', 'upper_bound': '8999'..."
1,!!!The government cannot fight CORRUPTION - th...,"[[{'percentage': '0.000933', 'age': '18-24', '...","[[{'percentage': '0.00185', 'region': 'Rhode I...",[{'lower_bound': '1000001'}],"[{'lower_bound': '0', 'upper_bound': '99'}]","[{'lower_bound': '1000', 'upper_bound': '1999'}]"
2,!!🚨Endorsement Alert🚨!! \n\nThank you Byron Po...,"[[{'percentage': '0.004039', 'age': '65+', 'ge...","[[{'percentage': '0.002281', 'region': 'Unknow...","[{'lower_bound': '100001', 'upper_bound': '500...","[{'lower_bound': '100', 'upper_bound': '199'}]","[{'lower_bound': '2000', 'upper_bound': '2999'}]"
3,""" An astute CPA told me 45 years ago, Math nev...","[[{'percentage': '0.00112', 'age': '18-24', 'g...","[[{'percentage': '1', 'region': 'Ohio'}], [{'p...","[{'lower_bound': '100001', 'upper_bound': '500...","[{'lower_bound': '0', 'upper_bound': '99'}, {'...","[{'lower_bound': '2000', 'upper_bound': '2999'..."
4,"""....Framingham should not go back to the ways...","[[{'percentage': '0.004255', 'age': '45-54', '...","[[{'percentage': '1', 'region': 'Massachusetts...","[{'lower_bound': '50001', 'upper_bound': '1000...","[{'lower_bound': '0', 'upper_bound': '99'}, {'...","[{'lower_bound': '0', 'upper_bound': '999'}, {..."


In [17]:
ind_region=df2_grouped.loc[6]['delivery_by_region']
ind_region

[[{'percentage': '1', 'region': 'Washington'}],
 [{'percentage': '1', 'region': 'Washington'}],
 [{'percentage': '1', 'region': 'Washington'}]]

In [18]:
ind_impressions=df2_grouped.loc[6]['impressions']
ind_impressions

[{'lower_bound': '0', 'upper_bound': '999'},
 {'lower_bound': '1000', 'upper_bound': '1999'},
 {'lower_bound': '0', 'upper_bound': '999'}]

In [29]:

for i in range(len(ind_region)):    
    region = ind_region[i]
    impr = ind_impressions[i]
    for key in region:    
        old = cur_cum_impressions[key['region']]
        region_name = key['region']
        cur_cum_impressions[region_name] = old+(float(impr['lower_bound']) + float(impr['upper_bound']))/2

In [30]:
cur_cum_impressions

defaultdict(int, {'Washington': 2498.5})

In [38]:
cum_impressions = []
for row in df2_grouped.iterrows():
    ind_region = row[1]['delivery_by_region']
    ind_impressions = row[1]['impressions']
    cur_cum_impressions = defaultdict(int)
    for i in range(len(ind_region)):    
        region = ind_region[i]
        impr = ind_impressions[i]
        for key in region:    
            old = cur_cum_impressions[key['region']]
            region_name = key['region']
            try: 
                cur_cum_impressions[region_name] = old+(float(impr['lower_bound']) + float(impr['upper_bound']))/2
            except:
                cur_cum_impressions[region_name] = old+(float(impr['lower_bound']))/2
            
    cum_impressions.append(cur_cum_impressions)

In [40]:
df2_grouped['cumulative_impressions_by_region'] = cum_impressions
df2_grouped.head()

,ad_creative_bodies,demographic_distribution,delivery_by_region,estimated_audience_size,spend,impressions,cumulative_impressions_by_region
0,,"[[{'percentage': '0.000876', 'age': '45-54', '...","[[{'percentage': '1', 'region': 'New York'}], ...","[{'lower_bound': '10001', 'upper_bound': '5000...","[{'lower_bound': '300', 'upper_bound': '399'},...","[{'lower_bound': '8000', 'upper_bound': '8999'...","{'New York': 28498.5, 'Georgia': 7999.0, 'Conn..."
1,!!!The government cannot fight CORRUPTION - th...,"[[{'percentage': '0.000933', 'age': '18-24', '...","[[{'percentage': '0.00185', 'region': 'Rhode I...",[{'lower_bound': '1000001'}],"[{'lower_bound': '0', 'upper_bound': '99'}]","[{'lower_bound': '1000', 'upper_bound': '1999'}]","{'Rhode Island': 1499.5, 'New York': 1499.5, '..."
2,!!🚨Endorsement Alert🚨!! \n\nThank you Byron Po...,"[[{'percentage': '0.004039', 'age': '65+', 'ge...","[[{'percentage': '0.002281', 'region': 'Unknow...","[{'lower_bound': '100001', 'upper_bound': '500...","[{'lower_bound': '100', 'upper_bound': '199'}]","[{'lower_bound': '2000', 'upper_bound': '2999'}]","{'Unknown': 2499.5, 'New Mexico': 2499.5}"
3,""" An astute CPA told me 45 years ago, Math nev...","[[{'percentage': '0.00112', 'age': '18-24', 'g...","[[{'percentage': '1', 'region': 'Ohio'}], [{'p...","[{'lower_bound': '100001', 'upper_bound': '500...","[{'lower_bound': '0', 'upper_bound': '99'}, {'...","[{'lower_bound': '2000', 'upper_bound': '2999'...",{'Ohio': 5999.0}
4,"""....Framingham should not go back to the ways...","[[{'percentage': '0.004255', 'age': '45-54', '...","[[{'percentage': '1', 'region': 'Massachusetts...","[{'lower_bound': '50001', 'upper_bound': '1000...","[{'lower_bound': '0', 'upper_bound': '99'}, {'...","[{'lower_bound': '0', 'upper_bound': '999'}, {...","{'Massachusetts': 999.0, 'Connecticut': 499.5,..."


In [41]:
df2_grouped.to_csv('emily.csv')